<a href="https://colab.research.google.com/github/prasa12/MyPython-Stuff/blob/master/Pytorch/01_Pytorch_Workflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pytorch Workflow

Lets explore Pytorch end to end workflow

In [ ]:
what_were_covering = {1:"data(prepare and load)",
                      2: "Build Model",
                      3: "Making predictions and evaluating a model(inference)",
                      4: "Save and loading the Model",
                      5: "Putting all together"}

what_were_covering

In [ ]:
import torch
from torch import nn
import matplotlib.pyplot as plt

# Check Pytorch versio
torch.__version__

from typing import Text
## 1. Data (preparing and loading)

Data can be almost anything in machine learning

* Excel Spreadsheet
* Images
* Videos
* Audio
* DNA
* Text

Machine is a game of 2 parts
1. Get data into numerical representation
2. Build a model to learn patterns in the Numerical representation

We will use a linear regression formula to make a straight line with known **parameters**

In [ ]:
# Create Known parameters
weight = 0.7
bias = 0.3

# Create
start = 0
end = 1
step = 0.02

X = torch.arange(start,end, step).unsqueeze(dim=1)
y = weight*X+bias

X[:10],y[:10]

In [ ]:
len(X),len(y)

In [ ]:
## Splitting the data into training and testing sets

# Create a train/test split
train_split = int(0.8*len(X))
X_train,y_train = X[:train_split], y[:train_split]
X_test, y_test = X[train_split:], y[train_split:]

len(X_train), len(y_train), len(X_test), len(y_test)


In [ ]:
def plot_predictions(train_data = X_train,
                     train_labels = y_train,
                     test_data = X_test,
                     test_labels = y_test,
                     predictions = None):
  """
  Plots training data, test data and compare predictions
  """
  plt.figure(figsize=(10,7))

  # Plot training data in blue
  plt.scatter(train_data, train_labels, c="b", s=4, label="Training data")

  # Plot test data in green
  plt.scatter(test_data,test_labels, c="g", s=4, label="Testing Data")

  # Are there predictions?
  if predictions is not None:
    #Plot predictions if they exist
    plt.scatter(test_data, predictions.detach().numpy(), c="r", s=4, label = "Predictions") # Detach and convert to numpy

  plt.legend(prop={"size": 14});

In [ ]:
plot_predictions()

## 2. Build a model
# Linear Regression model

In [ ]:
class LinearRegressionModel(nn.Module):
  def __init__(self):
    super().__init__()
    self.weights = nn.Parameter(torch.randn(1,
                                            requires_grad=True,
                                            dtype=torch.float))
    self.bias = nn.Parameter(torch.randn(1,
                                         requires_grad=True,
                                         dtype = torch.float))

    # Forward method to define the computation in the model
  def forward(self,x:torch.Tensor)->torch.Tensor:
    return self.weights*x+self.bias

### Pytorch Model Building essentials
## torch.nn - contains all the buildings for the computational graphs (a neural network can be considered a computational graph)
## torch.nn.Parameter - what parameters should our model try and learn, often a Pytorch layer from torch.nn will set these for us
## torch.nn.Module - The base class for all neural network modules, if you subclass it, you should overwrite forward()
## torch.optim - This is where the optimizers in the Pytorch live, they will help with gradient descent
## def forward() - All nn.Module subclasses require you to overwrite forward(), this method defines what happens in the forward computation



In [ ]:
### Checking the contents of the Pytorch model
torch.manual_seed(42)
model_0 = LinearRegressionModel()
list(model_0.parameters())

In [ ]:
model_0.state_dict()

In [ ]:
weight, bias

## Making predictions using 'torch.inference_mode()

To check out model's predictive power, let's see how well it predicts

In [ ]:
X_test,y_test

In [216]:
with torch.inference_mode():
  y_preds = model_0(X_test)

y_preds


tensor([[0.8685],
        [0.8825],
        [0.8965],
        [0.9105],
        [0.9245],
        [0.9384],
        [0.9524],
        [0.9664],
        [0.9804],
        [0.9944]])

In [ ]:
# inference_mode is preferred
with torch.no_grad():
  y_preds = model_0(X_test)

y_preds

In [ ]:
y_test


In [ ]:
plot_predictions(predictions=y_preds)

In [ ]:
## 3. Train Model
list(model_0.parameters())

In [ ]:
model_0.state_dict()

In [ ]:
# Set up a loss function and optimizer
loss_fn = nn.L1Loss()

optimizer = torch.optim.SGD(model_0.parameters(), lr=0.01)

### Building a traing loop in Pytorch
Things we need in training loop
0. Loop through Data
1. Forward pass
2. Calculate the Loss
3. optimizer
4. Loss Backward - moves backwards through the network and calculates the gradients
5. Optimizer step - use the optimizer parameters to optimize the step

In [ ]:
# An epoch is one loop through the data ...
epochs = 200

# Track different values
epoch_count = []
loss_values = []
test_loss_values = []

# 0. Loop through the data
for epoch in range(epochs):
  # Set the model to training mode
  model_0.train()

  # Forward pass
  y_pred = model_0(X_train)

  # Calculate the loss
  loss = loss_fn(y_pred, y_train)

  # Optimizer
  optimizer.zero_grad()

  # Loss backward
  loss.backward()

  # Optimizer step
  optimizer.step()

    # Evaluate the model on the test set
  model_0.eval() # Set the model to evaluation mode
  with torch.inference_mode():
    # Make predictions on the test data
    test_preds = model_0(X_test)

    # Calculate the loss on the test predictions
    test_loss = loss_fn(test_preds, y_test)

  if epoch % 10 == 0:
    epoch_count.append(epoch)
    loss_values.append(loss.item())
    test_loss_values.append(test_loss.item())

  print(f"Test loss: {test_loss.item():.4f}")

list(model_0.parameters())


In [ ]:
y_pred
loss, test_loss

In [ ]:
# Plot the loss curve
import numpy as np

plt.plot(epoch_count, np.array(torch.tensor(loss_values).numpy()) ,label = "Train Loss")
plt.plot(epoch_count, np.array(torch.tensor(test_loss_values).numpy()), label = "Test Loss")
plt.title("Training and Test Loss Curves")
plt.ylabel("Loss")
plt.xlabel("Epochs")
plt.legend()

In [ ]:
plot_predictions(predictions=test_preds)

## Saving a model in Pytorch
1. torch.save() - allows to save a Pytorch object in Python's pickle format
2. torch.load() - allows you load a saved Python object
3. torch.nn.Module.load_state_dict() - allows you to load the parameters of a Pytorch model

In [ ]:
from pathlib import Path

# Create models directory
MODEL_PATH = Path("models")
MODEL_PATH.mkdir(parents=True, exist_ok=True)
MODEL_NAME = "01_pytorch_workflow_model_0.pth"
MODEL_SAVE_PATH = MODEL_PATH / MODEL_NAME
MODEL_PATH, MODEL_SAVE_PATH
torch.save(obj=model_0.state_dict(),
           f=MODEL_SAVE_PATH)

In [207]:
model_0

LinearRegressionModel()

In [208]:
!ls -l models


total 4
-rw-r--r-- 1 root root 2117 Sep  3 06:15 01_pytorch_workflow_model_0.pth


In [210]:
# Loading a Pytorch Model
model_0.state_dict()

OrderedDict([('weights', tensor([0.6990])), ('bias', tensor([0.3093]))])

In [213]:
loaded_model_0 = LinearRegressionModel()
loaded_model_0.load_state_dict(torch.load(f=MODEL_SAVE_PATH))
loaded_model_0.state_dict()


OrderedDict([('weights', tensor([0.6990])), ('bias', tensor([0.3093]))])

In [214]:
# Make some predictions with the loaded model
with torch.inference_mode():
  loaded_model_preds = loaded_model_0(X_test)

loaded_model_preds

tensor([[0.8685],
        [0.8825],
        [0.8965],
        [0.9105],
        [0.9245],
        [0.9384],
        [0.9524],
        [0.9664],
        [0.9804],
        [0.9944]])

In [217]:
y_preds == loaded_model_preds

tensor([[True],
        [True],
        [True],
        [True],
        [True],
        [True],
        [True],
        [True],
        [True],
        [True]])